In [1]:
!pip install pythainlp


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pythainlp

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout
from tensorflow.keras.models import Sequential

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.metrics import classification_report

In [3]:
data = pd.read_excel('preprocessed_course_review.xlsx')

In [4]:
data.head(5)

,Unnamed: 0,text,score,course_name,score_encoded,Label_Enc
0,0,โค้ชเอ๋แน่นมากทั้ง MM Mind Set และ Method เทีย...,5,ทำกำไรในตลาดทองอย่างยั่งยืนด้วยเทคนิค 3M,Good,2
1,1,เยี่ยมเลยครับ คุณเอ๋ทำ tfex ด้วยนะครับ รอนะครับ,5,ทำกำไรในตลาดทองอย่างยั่งยืนด้วยเทคนิค 3M,Good,2
2,2,พี่เอ๋ ใช้เทคนิคง่ายไม่ซับซ้อน ดีครับ,5,ทำกำไรในตลาดทองอย่างยั่งยืนด้วยเทคนิค 3M,Good,2
3,3,ได้แนวคิดดีมากเลยค่ะ,5,ทำกำไรในตลาดทองอย่างยั่งยืนด้วยเทคนิค 3M,Good,2
4,4,โค้ชเอ๋ ถ่ายทอดเป็นระบบ ตามขั้นตอน ชัดเจน เข้า...,5,ทำกำไรในตลาดทองอย่างยั่งยืนด้วยเทคนิค 3M,Good,2


In [5]:
X = data["text"]
y = data["Label_Enc"]
y = to_categorical(y, num_classes=3)

In [6]:
X = [pythainlp.word_tokenize(text, engine='newmm') for text in X]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

X = pad_sequences(X, maxlen=50, padding='post')

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128, input_length=50))
lstm_model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
lstm_model.add(Dense(3, activation='softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-03-10 09:06:48.594399: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-10 09:06:48.595188: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
cnn_model = Sequential()
cnn_model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128, input_length=50))
cnn_model.add(Conv1D(64, 5, activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=4))
cnn_model.add(Conv1D(128, 5, activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=4))
cnn_model.add(GlobalMaxPooling1D())
cnn_model.add(Dense(3, activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])


In [10]:
from sklearn.utils.class_weight import compute_class_weight

# convert y_train to integer labels
y_train_int = np.argmax(y_train, axis=-1)

class_weights = compute_class_weight(
                   class_weight = "balanced",
                   classes = np.unique(y_train_int),
                   y = y_train_int
               )
class_weights = dict(enumerate(class_weights))

In [11]:
history = lstm_model.fit(X_train, y_train, batch_size=16, epochs=10, validation_split=0.2, class_weight=class_weights)
lstm_preds = np.argmax(lstm_model.predict(X_test), axis=-1)
print("LSTM Classification Report:")
print(classification_report(np.argmax(y_test, axis=-1), lstm_preds))

2023-03-10 09:06:49.053076: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/10


2023-03-10 09:06:50.244679: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


: 

: 

In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
history = cnn_model.fit(X_train, y_train, batch_size=16, epochs=10, validation_split=0.2)
cnn_preds = np.argmax(cnn_model.predict(X_test), axis=-1)
print("LSTM Classification Report:")
print(classification_report(np.argmax(y_test, axis=-1), cnn_preds))

In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()